# wee_slack sessionToken and d-cookie extraction

Harvesting the session token and the d-cookie (on any browser):

1. Start the web browser
1. Log on to Slack (can use SSO via SAML)
1. Copy the cookies and web storage database files to this notebook's work area
1. Run the notebook code for extracting the data from the corresponding browser

The d-cookie only exists while the browser is running because it's `HttpOnly`, set by the server at the beginning of the session.  Its value will be missing if the cookies database is harvested _after_ the Slack browser session closes.

---
## Firefox

Path:  `/Users/joeuser/Library/Application Support/Firefox/Profiles/xxxxxxxx.default-release`

where:  xxxxxxxx is generated by Firefox and varies from user to user.

### d-cookie extraction

In [ ]:
FIREFOX_COOKIES = 'cookies.sqlite'
FIREFOX_STORE = 'webappsstore.sqlite'

In [ ]:
import sqlite3

c = sqlite3.connect(FIREFOX_COOKIES)

In [ ]:
dCookie = c.execute('SELECT name, value, host FROM moz_cookies WHERE host = ".slack.com" AND name = "d"').fetchone()[1]

c.close()

### Session token extraction

In [ ]:
c = sqlite3.connect(FIREFOX_STORE)
rawConfig = c.execute("SELECT key, value FROM webappsstore2 WHERE key = 'localConfig_v2'").fetchone()[1]

c.close()

In [ ]:
import json

x = json.loads(rawConfig)

for config in (config for _, config in x['teams'].items()):
    if 'token' in config:
        sessionToken = config['token']
        break

### Results

Truncated `sessionToken` and `d-cookie` for privacy, visible only in the example command:

In [ ]:
print('/slack register %s:%s' % (sessionToken[:-8], dCookie[:-8]))

---
## Chrome

Path:  /Users/joeuser/Library/Application Support/Google/Chrome/Default

### d-cookie extraction

The d-cookie appears to be encrypted! 

In [ ]:
import sqlite3

c = sqlite3.connect('Cookies')

In [ ]:
p = c.execute("SELECT host_key, name, value FROM cookies WHERE host_key = '.slack.com' AND name = 'd'")

In [ ]:
for row in p:
    print(row)

In [ ]:
p.close()
c.close()

### Session token extraction

The session token is stored in LevelDB format.

In [ ]:
# Coming soon